In [1]:
import random

# Definição do grafo e entregas
grafo_conexoes = {
    'A': {'B': 5, 'D': 2},
    'B': {'C': 3, 'A': 5},
    'C': {'B': 3, 'D': 8},
    'D': {'A': 2, 'C': 8},
}

entregas = [
    {'inicio': 0, 'destino': 'B', 'bonus': 1},
    {'inicio': 5, 'destino': 'C', 'bonus': 10},
    {'inicio': 10, 'destino': 'D', 'bonus': 8},
]

def buscar_distancia(grafo, no_origem, no_destino):
    stack = [(no_origem, 0, set())]
    menor_distancia = float('inf')

    while stack:
        no_atual, distancia_atual, visitados = stack.pop()

        if no_atual == no_destino:
            menor_distancia = min(menor_distancia, distancia_atual)
            continue

        visitados.add(no_atual)

        for vizinho, distancia in grafo[no_atual].items():
            if vizinho not in visitados:
                stack.append((vizinho, distancia_atual + distancia, visitados.copy()))

    return menor_distancia if menor_distancia != float('inf') else None

# Parâmetros do ACO
num_formigas = 10
num_iteracoes = 50
evaporacao = 0.5
intensidade_feromona = 1.0

# Inicialização da matriz de feromona
feromona = {(origem, destino): 1.0 for origem in grafo_conexoes for destino in grafo_conexoes[origem]}

def calcular_probabilidade(feromona, distancias, origem, destino):
    return feromona.get((origem, destino), 0.1) / distancias.get((origem, destino), 1.0)

def aco():
    melhor_bônus = 0
    melhor_solução = []

    for iteracao in range(num_iteracoes):
        todas_solucoes = []

        for _ in range(num_formigas):
            solucao_atual = []
            tempo_atual = 0
            bonus_total = 0
            entregas_restantes = entregas[:]

            for _ in range(len(entregas)):
                if not entregas_restantes:
                    break

                entrega = random.choice(entregas_restantes)
                destino = entrega['destino']
                inicio = entrega['inicio']
                bonus = entrega['bonus']

                if tempo_atual <= inicio:
                    distancia = buscar_distancia(grafo_conexoes, 'A', destino)
                    if distancia is not None:
                        tempo_atual = inicio + distancia * 2
                        bonus_total += bonus
                        solucao_atual.append(entrega)
                        entregas_restantes.remove(entrega)

                        distancia_retorno = buscar_distancia(grafo_conexoes, destino, 'A')
                        if distancia_retorno is not None:
                            tempo_atual += distancia_retorno

            if bonus_total > melhor_bônus:
                melhor_bônus = bonus_total
                melhor_solução = solucao_atual

            todas_solucoes.append((solucao_atual, bonus_total))

        # Atualiza feromona
        for (solucao, bonus) in todas_solucoes:
            for entrega in solucao:
                origem = 'A'
                destino = entrega['destino']
                if (origem, destino) in feromona:
                    feromona[(origem, destino)] += intensidade_feromona * bonus
                    feromona[(origem, destino)] *= (1 - evaporacao)

    return melhor_solução, melhor_bônus

# Executa o ACO
solucao_final, bonus_final = aco()
print(f'Solução final: {solucao_final}')
print(f'Bônus total: {bonus_final}')

Solução final: [{'inicio': 5, 'destino': 'C', 'bonus': 10}]
Bônus total: 10
